In [1]:
from pathlib import Path
from ultralytics import YOLO
import cv2
import shutil
import yaml

In [ ]:
def create_dataset(dataset_path, output_dir="cubesat_dataset"):
    """Создаем датасет для обучения"""
    data_path = Path(dataset_path)
    dataset_dir = Path(output_dir)

    # Создание структуры директорий
    for folder in ["images/train", "images/val", "labels/train", "labels/val"]:
        (dataset_dir / folder).mkdir(parents=True, exist_ok=True)

    # Загрузка предобученной модели для авторазметки
    model = YOLO('yolov8n.pt')

    # Получение тренировочных изображений
    train_images = list((data_path / "train" / "images").glob("*.jpg"))
    print(f"Обрабатываем {len(train_images)} тренировочных изображений...")

    for img_path in train_images:
        # Копирование изображения
        dst = dataset_dir / "images" / "train" / img_path.name
        shutil.copy(img_path, dst)

        # Детекция кубсатов с низким порогом уверенности (0.1)
        results = model(img_path, conf=0.1, verbose=False)

        with open(dataset_dir / "labels" / "train" / f"{img_path.stem}.txt", 'w') as f:
            if results[0].boxes is not None:
                img = cv2.imread(str(img_path))
                h, w = img.shape[:2]

                for box in results[0].boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    # Конвертация координат в YOLO-формат (нормализованные центры)
                    x_center = ((x1 + x2) / 2) / w
                    y_center = ((y1 + y2) / 2) / h
                    width = (x2 - x1) / w
                    height = (y2 - y1) / h

                    # Фильтрация слишком маленьких объектов
                    if width > 0.01 and height > 0.01:
                        f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    # Аналогичная обработка валидационных изображений
    val_images = list((data_path / "val" / "images").glob("*.jpg"))
    print(f"Обрабатываем {len(val_images)} валидационных изображений...")

    for img_path in val_images:
        dst = dataset_dir / "images" / "val" / img_path.name
        shutil.copy(img_path, dst)

        results = model(img_path, conf=0.1, verbose=False)

        with open(dataset_dir / "labels" / "val" / f"{img_path.stem}.txt", 'w') as f:
            if results[0].boxes is not None:
                img = cv2.imread(str(img_path))
                h, w = img.shape[:2]

                for box in results[0].boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    x_center = ((x1 + x2) / 2) / w
                    y_center = ((y1 + y2) / 2) / h
                    width = (x2 - x1) / w
                    height = (y2 - y1) / h

                    if width > 0.01 and height > 0.01:
                        f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    # Создание YAML-конфигурации датасета
    config = {
        'path': str(dataset_dir.absolute()),  # Абсолютный путь к датасету
        'train': 'images/train',              # Путь к тренировочным изображениям
        'val': 'images/val',                  # Путь к валидационным изображениям
        'nc': 1,                              # Количество классов
        'names': ['cubesat']                  # Имена классов
    }

    yaml_path = dataset_dir / "dataset.yaml"
    with open(yaml_path, 'w') as f:
        yaml.dump(config, f)

    return yaml_path

In [3]:
def train_model(data_yaml):
    """Обучаем модель"""
    model = YOLO('yolov8n.pt')

    results = model.train(
        data=str(data_yaml),  # Путь к конфигурации датасета
        epochs=50,            # Количество эпох обучения
        imgsz=640,            # Размер входного изображения
        batch=16,             # Размер батча (оптимально для 8GB VRAM)
        device='cuda',        # Использование GPU
        workers=4,            # Количество процессов загрузки данных
        amp=True,             # Mixed precision training для экономии памяти
        lr0=0.01,             # Начальная скорость обучения
        augment=True,         # Аугментация данных
        save=True,            # Сохранение чекпоинтов
        plots=True,           # Генерация графиков обучения
        val=True,             # Валидация после каждой эпохи
        exist_ok=True,        # Перезапись существующих runs
        pretrained=True,      # Использование предобученных весов
        verbose=True,         # Подробный вывод процесса
    )

    # Сохранение лучшей модели в корень проекта
    if Path('runs/detect/train/weights/best.pt').exists():
        shutil.copy('runs/detect/train/weights/best.pt', 'best.pt')
        print("\nМодель сохранена как 'best.pt'")

    return results

In [4]:
def main():
    # Путь к исходному датасету
    dataset_path = r"C:\\Users\\ASUS\\.cache\\kagglehub\\datasets\\eberhardtkorf\\synthetic-cubesat\\versions\\1\\synthetic_cubesat\\dataset"

    print("Создание датасета...")
    data_yaml = create_dataset(dataset_path)

    print("\nОбучение модели...")
    train_model(data_yaml)

    print("\nОбучение завершено")

In [5]:
if __name__ == "__main__":
    main()

Создание датасета...
Обрабатываем 6798 тренировочных изображений...
Обрабатываем 1200 валидационных изображений...

Обучение модели...
Ultralytics 8.4.8  Python-3.11.4 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=cubesat_dataset\dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic